In [43]:
import pandas as pd
import datetime
import gzip
import io

---

## <center>Funciones</center>

In [109]:
def extraer_fecha(time):
    time_seconds = time / 1000  # Convertir milisegundos a segundos
    fecha_hora = datetime.datetime.utcfromtimestamp(time_seconds)

    # Eliminar los milisegundos
    fecha_hora = fecha_hora.replace(microsecond=0)

    return fecha_hora


In [110]:
def extraer_fecha_opinion(time):
    
    time = int(time)
    
    time_seconds = time / 1000  # Convertir milisegundos a segundos
    
    fecha_hora = datetime.datetime.utcfromtimestamp(time_seconds)

    # Eliminar los milisegundos
    fecha_hora = fecha_hora.replace(microsecond=0)

    return fecha_hora

---

In [111]:
json = pd.read_json(r'datasets\Google Maps\reviews-estados\review-Alabama\1.json', lines=True)

In [112]:
''' Elimino la columna pics, las imagenes son muy chicas y no tienen relevancia para su proximo analisis, en cualquier caso si es necesario acceder a las mismas, una alternativa mucho mejor es utilizar la api o scraping con su id'''
json.drop(columns=['pics'],inplace=True) 

In [113]:
''' Utilizo la funcion creada anteriormente, para convertir los milisegundos a una fecha entendible, luego lo separo en 2 columnas para poder comparar el tiempo de respuesta con el de la opinion propiamente dicha'''
json['time'] = json['time'].apply(extraer_fecha)

In [114]:
'''

Desanido la columna resp, para luego poder realizar calculos con el tiempo de respuesta y de ser necesario el analisis de sentimiento de la respuesta por separado.

'''
json['fecha_respuesta'] = None
json['respuesta'] = None

# Desanidar la columna 'resp' si no es None y asignar valores a las nuevas columnas
json.loc[json['resp'].notnull(), 'fecha_respuesta'] = json.loc[json['resp'].notnull(), 'resp'].apply(lambda x: x.get('time') if x else None)
json.loc[json['resp'].notnull(), 'respuesta'] = json.loc[json['resp'].notnull(), 'resp'].apply(lambda x: x.get('text') if x else None)

# Convertir la columna 'respuesta' a tipo de datos 'object' para permitir listas o strings
json['respuesta'] = json['respuesta'].astype('object')

# Luego, realizar la asignación de valores a la columna 'respuesta'
json.loc[json['resp'].notnull(), 'respuesta'] = json.loc[json['resp'].notnull(), 'resp'].apply(lambda x: x.get('text') if x else None)

json.drop(columns=['resp'],inplace=True)

In [116]:
json['fecha_respuesta'].fillna(json['time'], inplace=True)

In [117]:
json['fecha_respuesta'] = json['fecha_respuesta'].astype(str)

In [118]:
# Aplicar la función 'extraer_anio()' a las filas que cumplen con la condición de longitud
json.loc[json['fecha_respuesta'].str.len() < 14, 'fecha_respuesta'] = json.loc[json['fecha_respuesta'].str.len() < 14, 'fecha_respuesta'].apply(extraer_fecha_opinion)
json['fecha_respuesta'] = pd.to_datetime(json['fecha_respuesta'], format='%Y-%m-%d %H:%M:%S')

In [134]:
json2 = json.copy()

json2['anio'] = json['fecha_respuesta'].dt.year  # Extraer el año y crear una nueva columna 'anio'
resultados = json2.groupby('anio').size()  # Agrupar por año y contar la cantidad de registros
resultados


anio
2008        1
2009        2
2010       10
2011       57
2012       95
2013      170
2014      264
2015      802
2016     4327
2017    18019
2018    30822
2019    41766
2020    29962
2021    23703
dtype: int64

In [135]:
json

,user_id,name,time,rating,text,gmap_id,fecha_respuesta,respuesta
0,1.140438e+20,Kanisha Mixon,2020-08-11 17:51:12,5,Very Personable staff! Beautiful and clean env...,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,2020-08-11 17:51:12,None
1,1.160090e+20,Brandie Hodges,2021-01-06 02:10:39,5,Best clothing intown,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,2021-01-06 02:10:39,None
2,1.062399e+20,Sharon King,2019-01-11 19:34:50,4,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,2019-01-11 19:34:50,None
3,1.049701e+20,Veronica Pierce,2018-02-04 01:56:43,5,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,2018-02-04 01:56:43,None
4,1.105875e+20,Whitney Waldon Collier,2018-08-26 01:08:38,5,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,2018-08-26 01:08:38,None
...,...,...,...,...,...,...,...,...
149995,1.165082e+20,Lee Lee,2020-10-02 19:21:29,5,Just a good old hardware friendly people,0x889a89fd92d55353:0x78e335f5e98e75b2,2020-10-02 19:21:29,None
149996,1.080208e+20,Hayward Salter,2021-05-26 02:03:33,5,Good people,0x889a89fd92d55353:0x78e335f5e98e75b2,2021-05-26 02:03:33,None
149997,1.054646e+20,James McBride,2020-10-05 02:00:28,4,Good for locals needing supplies to fix damage...,0x889a89fd92d55353:0x78e335f5e98e75b2,2020-10-05 02:00:28,None
149998,1.118869e+20,Karen Sullivan,2020-06-24 15:41:25,1,Very rude female employees at the cash counter...,0x889a89fd92d55353:0x78e335f5e98e75b2,2020-06-24 15:41:25,None
